In [1]:
import spacy
from spacy.symbols import ORTH
from torchtext import data, datasets
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
print("Use CUDA:", USE_CUDA)

Use CUDA: False


# Hyperparameters

In [3]:
nhid = 200
embed_dim = 300
lr = 10

# Read Data

In [4]:
bptt_len = 60
batch_size = 32

In [5]:
text_file = 'combined.txt'

In [6]:
lm_tok = spacy.load('en')
def spacy_tok(x):
    return [tok.text for tok in lm_tok.tokenizer(x)]

TEXT = data.ReversibleField(sequential=True, tokenize=spacy_tok,
                            lower=True, include_lengths=False)

In [7]:
text_dataset = datasets.LanguageModelingDataset(text_file, TEXT, newline_eos=True)

In [8]:
vectors = "glove.840B.300d"
TEXT.build_vocab(questions_dataset, vectors=vectors)

NameError: name 'questions_dataset' is not defined

In [24]:
# iterators
train_iter = data.BPTTIterator(train_dataset, batch_size=batch_size, bptt_len=bptt_len, repeat=False, shuffle=True)
dev_iter = data.BPTTIterator(dev_dataset, batch_size=batch_size, bptt_len=bptt_len, repeat=False)

In [25]:
len(TEXT.vocab)

15871

In [27]:
# Create embeddings
embedding = nn.Embedding(len(TEXT.vocab), embed_dim)
embedding.weight.data.copy_(TEXT.vocab.vectors)
embedding.weight.requires_grad = False
embedding = embedding.to(device)

# Define Model

In [30]:
class LM(nn.Module):
    def __init__(self, ntoken, ninp, nhid, embedding, dropout=0.5):
        super(LM, self).__init__()
        self.nhid = nhid
        self.encoder = embedding
        self.rnn = nn.LSTM(ninp, nhid, batch_first=True)
        self.decoder = nn.Linear(nhid, ntoken)
        self.embed_drop = nn.Dropout(dropout)
        self.output_drop = nn.Dropout(dropout)
#         self.embed_drop = LockedDropout(dropout)
#         self.output_drop = LockedDropout(dropout)

        # # tie weights
        # self.decoder.weight = self.encoder.weight

    def forward(self, inputs, hidden=None):
        """

        :param inputs: (batch_size, max_len)
        :param hidden: ((1, batch_size, nhid), (1, batch_size, nhid))
        :return:
        """
        emb = self.embed_drop(self.encoder(inputs))
        if hidden:
            outputs, hidden = self.rnn(emb, hidden)
        else:
            outputs, hidden = self.rnn(emb)
        outputs = self.output_drop(outputs)
        decoded = self.decoder(outputs)
        return decoded, outputs, hidden

In [33]:
lm = LM(len(TEXT.vocab), embed_dim, nhid, embedding)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(lm.parameters(), lr=lr)

# Training

In [36]:
for batch in bpttiter:
    x, y = batch.text.transpose(0, 1).contiguous().to(device), \
                   batch.target.transpose(0, 1).contiguous().to(device)
    print(x.shape)
    print(y.shape)
    
    out, _, _ = lm(x)
    print(out.shape)
    
#     print(x)
#     print(y)
    break

torch.Size([32, 60])
torch.Size([32, 60])
torch.Size([32, 60, 15871])
